# **Dự Đoán Giá Nhà từ Dữ Liệu Bất Động Sản ở Hà Nội (Tuần 2)**

Ở bài trước, chúng ta đã chạy được lan truyền xuôi dựa trên tập dữ liệu bất động sản ở Hà Nội. Trong bài tập này, chúng ta sẽ đi sâu hơn về lan truyền ngược. Trong phạm vi của lan truyền ngược, chúng ta chỉ viết các hàm (functions), và chạy qua 1 lượt lan truyền ngược mẫu, phần huấn luyện mô hình còn lại sẽ nằm ở tuần 3.

**Dữ Liệu**:

Chúng ta sẽ sử dụng tập dữ liệu về dự đoán giá nhà được lấy từ Kaggle (https://www.kaggle.com/datasets/ladcva/vietnam-housing-dataset-hanoi/data). Tập dữ liệu này chứa thông tin về các giao dịch bất động sản ở Hà Nội, bao gồm các thuộc tính liên quan như địa chỉ, quận, huyện, diện tích, số phòng ngủ,... và giá bán. Mục tiêu của chúng ta là xây dựng một mô hình để dự đoán giá của các căn nhà dựa trên các thuộc tính này.

**Phương Pháp**:

Chúng ta sẽ tiến hành các bước sau:
* Hàm mất mát MSE
* Lan truyền ngược

In [9]:
import numpy as np

# BÀI TẬP: Tính độ mất mát
Trong cell này, chúng ta định nghĩa hàm compute_cost để tính toán độ mất mát của mô hình bằng phương pháp Mean Squared Error (MSE).

## Giải thích Mean Squared Error (MSE):

Mean Squared Error là trung bình của tổng các bình phương của sự khác biệt giữa giá trị dự đoán $A$ và giá trị thực tế $Y$.
Công thức:
\begin{equation*}
\text{MSE} = \frac{1}{2m} \sum_{i=1}^{m} (A_i - Y_i)^2
\end{equation*}

Trong đó:
- $m$ là số lượng ví dụ.
- $A_i$ là giá trị dự đoán cho ví dụ thứ $i$.
- $Y_i$ là giá trị thực cho ví dụ thứ $i$.
2

Gợi ý: sử dụng hàm [numpy.sum](https://numpy.org/doc/2.0/reference/generated/numpy.sum.html) và [numpy.square](https://numpy.org/doc/stable/reference/generated/numpy.square.html)

In [1]:
# Tính toán độ mất mát (mean squared error)
def compute_cost(A, Y):
    """
    Tính toán lỗi bình phương trung bình.

    Tham số:
    A -- Đầu ra của kích hoạt cuối cùng, có dạng (1, số lượng ví dụ)
    Y -- vector nhãn "thật" có dạng (1, số lượng ví dụ)

    Trả về:
    cost -- lỗi bình phương trung bình
    """
    m = Y.shape[1]
    ### BEGIN SOLUTION
    cost = (1.0 / (2 * m)) * np.sum(np.square(A - Y))
    ### END SOLUTION
    return float(np.squeeze(cost))

In [3]:
# Tests 10 points.
# Test Case 1: Perfect predictions
def test_case_1():
    Y = np.array([[1, 0, 1, 1, 0]])
    A = np.array([[1, 0, 1, 1, 0]])
    cost = compute_cost(A, Y)
    assert np.isclose(cost, 0), f"Cost should be 0 for perfect predictions, but got {cost}"
    print("Test Case 1 passed!")

test_case_1()

### BEGIN HIDDEN TESTS
# Test Case 2: Completely wrong predictions
def test_case_2():
    Y = np.array([[1, 0, 1, 1, 0]])
    A = np.array([[0, 1, 0, 0, 1]])
    cost = compute_cost(A, Y)
    assert np.isclose(cost, 0.5), f"Cost should be 0.5 for completely wrong predictions, but got {cost}"
    print("Test Case 2 passed!")

# Test Case 3: Mixed predictions
def test_case_3():
    Y = np.array([[1, 0, 1, 1, 0]])
    A = np.array([[0.8, 0.2, 0.7, 0.9, 0.1]])
    expected_cost = 0.019000000000000003
    cost = compute_cost(A, Y)
    assert np.isclose(cost, expected_cost, atol=1e-4), f"Expected cost {expected_cost}, but got {cost}"
    print("Test Case 3 passed!")

# Test Case 4: Large input
def test_case_4():
    np.random.seed(42)
    Y = np.random.randint(0, 2, (1, 1000))
    A = np.random.rand(1, 1000)
    cost = compute_cost(A, Y)
    assert 0 <= cost <= 0.5, f"Cost should be between 0 and 0.5, but got {cost}"
    print("Test Case 4 passed!")

# Test Case 5: Edge case - all zeros
def test_case_5():
    Y = np.zeros((1, 100))
    A = np.zeros((1, 100))
    cost = compute_cost(A, Y)
    assert np.isclose(cost, 0), f"Cost should be 0 for all zeros, but got {cost}"
    print("Test Case 5 passed!")

# Run all test cases
test_case_2()
test_case_3()
test_case_4()
test_case_5()
### END HIDDEN TESTS

Test Case 1 passed!
Test Case 2 passed!
Test Case 3 passed!
Test Case 4 passed!
Test Case 5 passed!


# BÀI TẬP: Lan truyền ngược
Trong cell này, chúng ta định nghĩa hàm backward_propagation để thực hiện lan truyền ngược cho một lớp của mạng neural. Lan truyền ngược giúp tính toán gradient của hàm mất mát đối với các tham số của mạng, qua đó cập nhật trọng số và bias để tối ưu hóa mô hình.

Đây là cách bạn có thể viết lại các công thức theo định dạng mà bạn yêu cầu:

### Giải thích Chi Tiết Lan Truyền Ngược Dựa Vào Chain Rule

#### Backward Propagation

Quá trình lan truyền ngược dựa vào quy tắc chuỗi để tính toán gradient của hàm mất mát $L$ với từng tham số $W$, $b$, và $A$.

##### Đạo Hàm Với Lớp Đầu Ra (Output Layer)

- **Tính $dZ^{[2]}$**
\begin{equation*}
   dZ^{[2]} = A^{[2]} - Y
\end{equation*}
 - Đây là gradient của hàm mất mát $L$ đối với $Z^{[2]}$. Nếu hàm mất mát là hàm bình phương (mean squared error), thì $dZ^{[2]} = A^{[2]} - Y$.

- **Tính $dW^{[2]}$**
\begin{equation*}
   dW^{[2]} = \frac{1}{m} dZ^{[2]} (A^{[1]})^T
\end{equation*}
 - Sử dụng quy tắc chuỗi, chúng ta có $dW^{[2]} = \frac{\partial L}{\partial W^{[2]}} = \frac{\partial L}{\partial Z^{[2]}} \cdot \frac{\partial Z^{[2]}}{\partial W^{[2]}}$.
 - $\frac{\partial Z^{[2]}}{\partial W^{[2]}} = A^{[1]}$, do đó $dW^{[2]} = \frac{1}{m} dZ^{[2]} (A^{[1]})^T$.

- **Tính $db^{[2]}$**
\begin{equation*}
   db^{[2]} = \frac{1}{m} \sum dZ^{[2]}
\end{equation*}
 - Gradient của hàm mất mát $L$ đối với bias $b^{[2]}$ là tổng của tất cả các gradient của $Z^{[2]}$ chia cho số lượng ví dụ $m$.

##### Đạo Hàm Với Lớp Ẩn (Hidden Layer)

- **Tính $dZ^{[1]}$**
\begin{equation*}
   dZ^{[1]} = W^{[2]T} dZ^{[2]} \odot g'^{[1]}(Z^{[1]})
\end{equation*}
 - Sử dụng quy tắc chuỗi, gradient của hàm mất mát $L$ đối với $Z^{[1]}$ là:
\begin{equation*}
dZ^{[1]} = \frac{\partial L}{\partial A^{[1]}} \cdot \frac{\partial A^{[1]}}{\partial Z^{[1]}}
\end{equation*}
 - $\frac{\partial A^{[1]}}{\partial Z^{[1]}} = g'^{[1]}(Z^{[1]})$.
 - $\frac{\partial L}{\partial A^{[1]}} = W^{[2]T} dZ^{[2]}$.

- **Tính $dW^{[1]}$**
\begin{equation*}
   dW^{[1]} = \frac{1}{m} dZ^{[1]} (X^T)
\end{equation*}
 - Sử dụng quy tắc chuỗi, $dW^{[1]} = \frac{\partial L}{\partial W^{[1]}} = \frac{\partial L}{\partial Z^{[1]}} \cdot \frac{\partial Z^{[1]}}{\partial W^{[1]}}$.
 - $\frac{\partial Z^{[1]}}{\partial W^{[1]}} = X$.

- **Tính $db^{[1]}$**
\begin{equation*}
   db^{[1]} = \frac{1}{m} \sum dZ^{[1]}
\end{equation*}
 - Tương tự như với lớp đầu ra, gradient của hàm mất mát $L$ đối với bias $b^{[1]}$ là tổng của tất cả các gradient của $Z^{[1]}$ chia cho số lượng ví dụ $m$.

### Tóm Tắt Bằng Công Thức

- **Tính $dZ^{[2]}$**
\begin{equation*}
   dZ^{[2]} = A^{[2]} - Y
\end{equation*}

- **Tính $dW^{[2]}$**
\begin{equation*}
   dW^{[2]} = \frac{1}{m} dZ^{[2]} (A^{[1]})^T
\end{equation*}

- **Tính $db^{[2]}$**
\begin{equation*}
   db^{[2]} = \frac{1}{m} \sum dZ^{[2]}
\end{equation*}

- **Tính $dZ^{[1]}$**
\begin{equation*}
   dZ^{[1]} = W^{[2]T} dZ^{[2]} \odot g'^{[1]}(Z^{[1]})
\end{equation*}

- **Tính $dW^{[1]}$**
\begin{equation*}
   dW^{[1]} = \frac{1}{m} dZ^{[1]} (X^T)
\end{equation*}

- **Tính $db^{[1]}$**
\begin{equation*}
   db^{[1]} = \frac{1}{m} \sum dZ^{[1]}
\end{equation*}

## Đạo hàm của ReLU

Hàm `relu_derivative(x)` được dùng để tính đạo hàm của hàm kích hoạt ReLU. Đối với các giá trị dương của đầu vào `x`, đạo hàm sẽ bằng 1, và với các giá trị không dương, đạo hàm sẽ bằng 0. Để đảm bảo tính linh hoạt khi xử lý cả giá trị đơn lẻ và mảng, hàm sử dụng [np.where](https://numpy.org/doc/2.1/reference/generated/numpy.where.html) để thực hiện phép kiểm tra điều kiện.

In [6]:
def relu_derivative(x):
    ### BEGIN SOLUTION
    # Use np.where to handle both scalars and arrays
    return np.where(x > 0, 1, 0)
    ### END SOLUTION

In [8]:
# Tests 10 points.

def relu_derivative(x):
    # Use np.where to handle both scalars and arrays
    return np.where(x > 0, 1, 0)

# Test Case 1: Basic scalar inputs
def test_case_1():
    assert relu_derivative(5.0) == 1, "Failed on positive scalar input"
    assert relu_derivative(-5.0) == 0, "Failed on negative scalar input"
    assert relu_derivative(0) == 0, "Failed on zero input"
    print("Test Case 1 passed!")

test_case_1()

### BEGIN HIDDEN TESTS

# Test Case 2: 1D array input
def test_case_2():
    x = np.array([-2, -1, 0, 1, 2])
    expected = np.array([0, 0, 0, 1, 1])
    result = relu_derivative(x)
    assert np.array_equal(result, expected), "Failed on 1D array input"
    print("Test Case 2 passed!")

# Test Case 3: 2D array input
def test_case_3():
    x = np.array([[-1, 0, 1], [2, -3, 4]])
    expected = np.array([[0, 0, 1], [1, 0, 1]])
    result = relu_derivative(x)
    assert np.array_equal(result, expected), "Failed on 2D array input"
    print("Test Case 3 passed!")

# Test Case 4: Float array input
def test_case_4():
    x = np.array([-0.5, 0.0, 0.5])
    expected = np.array([0, 0, 1])
    result = relu_derivative(x)
    assert np.array_equal(result, expected), "Failed on float array input"
    print("Test Case 4 passed!")

# Test Case 5: Large values
def test_case_5():
    x = np.array([-1e6, 1e6])
    expected = np.array([0, 1])
    result = relu_derivative(x)
    assert np.array_equal(result, expected), "Failed on large values"
    print("Test Case 5 passed!")

# Test Case 6: Small values
def test_case_6():
    x = np.array([-1e-6, 1e-6])
    expected = np.array([0, 1])
    result = relu_derivative(x)
    assert np.array_equal(result, expected), "Failed on small values"
    print("Test Case 6 passed!")

# Test Case 7: Empty array
def test_case_7():
    x = np.array([])
    result = relu_derivative(x)
    assert len(result) == 0, "Failed on empty array length check"
    assert isinstance(result, np.ndarray), "Failed on empty array type check"
    print("Test Case 7 passed!")

# Test Case 8: Broadcasting behavior
def test_case_8():
    x = np.array([[1, -1], [-1, 1]])
    expected = np.array([[1, 0], [0, 1]])
    result = relu_derivative(x)
    assert np.array_equal(result, expected), "Failed on broadcasting test"
    print("Test Case 8 passed!")

test_case_2()
test_case_3()
test_case_4()
test_case_5()
test_case_6()
test_case_7()
test_case_8()

### END HIDDEN TESTS

Test Case 1 passed!
Test Case 2 passed!
Test Case 3 passed!
Test Case 4 passed!
Test Case 5 passed!
Test Case 6 passed!
Test Case 7 passed!
Test Case 8 passed!


Hàm `backward_propagation(dA, cache, activation)` thực hiện lan truyền ngược qua một lớp của mạng neural, giúp tính toán các gradient của hàm mất mát đối với các tham số của lớp đó (trọng số và bias), cũng như gradient đối với đầu vào của lớp. Đây là bước cần thiết để cập nhật các tham số trong quá trình huấn luyện.

Trong hàm này:
- `dA` là gradient của hàm mất mát đối với đầu ra của lớp hiện tại.
- `cache` chứa các giá trị trung gian từ lan truyền xuôi, bao gồm đầu vào, trọng số, và đầu ra trước khi áp dụng hàm kích hoạt.
- `activation` là loại hàm kích hoạt, có thể là `relu` hoặc `linear`, với các cách tính gradient khác nhau:
  - Nếu hàm kích hoạt là `relu`, gradient được nhân với đạo hàm của ReLU.
  - Nếu là `linear`, thì gradient đầu ra (`dZ`) giữ nguyên giá trị của `dA`.

In [4]:
# Lan truyền ngược (một lớp)
def backward_propagation(dA, cache, activation):
    """
    Thực hiện lan truyền ngược qua một lớp của mạng neural.

    Hàm này tính toán gradient của hàm mất mát đối với các tham số của lớp (trọng số và bias)
    và gradient đối với đầu vào của lớp đó. Đây là bước cần thiết để cập nhật các tham số trong quá trình huấn luyện.

    Tham số:
    dA -- Gradient của hàm mất mát đối với đầu ra của lớp hiện tại, có kích thước (số lượng đơn vị lớp hiện tại, số lượng ví dụ).
    cache -- Dictionary chứa các giá trị cần thiết cho lan truyền ngược, bao gồm:
        - "Z" -- Giá trị trước kích hoạt của lớp hiện tại.
        - "A_prev" -- Đầu vào của lớp hiện tại.
        - "W" -- Trọng số của lớp hiện tại.
        - "A" -- Đầu ra của lớp hiện tại sau khi áp dụng hàm kích hoạt.
    activation -- Loại hàm kích hoạt, có thể là "relu" hoặc "linear".

    Trả về:
    dA_prev -- Gradient của hàm mất mát đối với đầu vào của lớp hiện tại, có kích thước (số lượng đặc trưng, số lượng ví dụ).
    dW -- Gradient của hàm mất mát đối với ma trận trọng số của lớp hiện tại.
    db -- Gradient của hàm mất mát đối với vector bias của lớp hiện tại.
    """
    Z = cache["Z"]

    """
    Gợi ý:

    if activation == "relu":
        ... # Nhân dA với đạo hàm của hàm ReLU
    elif activation == "linear":
        ...  # Đối với kích hoạt tuyến tính, dZ = dA
    else:
        raise ValueError("Hàm kích hoạt không được hỗ trợ")

    A_prev = cache["A_prev"]
    m = ... # Số lượng data của A_prev

    dW = ... # Dựa trên công thức ở trên
    db = (1 / m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(cache["W"].T, dZ)
    """

    ### BEGIN SOLUTION
    if activation == "relu":
        dZ = dA * relu_derivative(Z)
    elif activation == "linear":
        dZ = dA  # Đối với kích hoạt tuyến tính, dZ = dA
    else:
        raise ValueError("Hàm kích hoạt không được hỗ trợ")

    A_prev = cache["A_prev"]
    m = A_prev.shape[1]

    dW = (1 / m) * np.dot(dZ, A_prev.T)
    db = (1 / m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(cache["W"].T, dZ)
    ### END SOLUTION

    return dA_prev, dW, db

In [7]:
# Tests 10 points.

def relu(Z):
    return np.maximum(0, Z)

# Test Case 1: ReLU activation
def test_case_1():
    np.random.seed(1)
    dA = np.random.randn(4, 2)
    A_prev = np.random.randn(3, 2)
    W = np.random.randn(4, 3)
    b = np.random.randn(4, 1)
    Z = np.dot(W, A_prev) + b
    cache = {"Z": Z, "A_prev": A_prev, "W": W, "A": relu(Z)}

    dA_prev, dW, db = backward_propagation(dA, cache, "relu")

    assert dA_prev.shape == (3, 2), f"dA_prev shape is incorrect. Expected (3, 2), but got {dA_prev.shape}"
    assert dW.shape == (4, 3), f"dW shape is incorrect. Expected (4, 3), but got {dW.shape}"
    assert db.shape == (4, 1), f"db shape is incorrect. Expected (4, 1), but got {db.shape}"

    # Manual calculation for comparison
    dZ = dA * relu_derivative(Z)
    expected_dW = (1 / 2) * np.dot(dZ, A_prev.T)
    expected_db = (1 / 2) * np.sum(dZ, axis=1, keepdims=True)
    expected_dA_prev = np.dot(W.T, dZ)

    assert np.allclose(dW, expected_dW), "dW calculation is incorrect"
    assert np.allclose(db, expected_db), "db calculation is incorrect"
    assert np.allclose(dA_prev, expected_dA_prev), "dA_prev calculation is incorrect"

    print("Test Case 1 passed!")

test_case_1()

### BEGIN HIDDEN TESTS

# Test Case 2: Linear activation
def test_case_2():
    np.random.seed(2)
    dA = np.random.randn(2, 3)
    A_prev = np.random.randn(4, 3)
    W = np.random.randn(2, 4)
    b = np.random.randn(2, 1)
    Z = np.dot(W, A_prev) + b
    cache = {"Z": Z, "A_prev": A_prev, "W": W, "A": Z}

    dA_prev, dW, db = backward_propagation(dA, cache, "linear")

    assert dA_prev.shape == (4, 3), f"dA_prev shape is incorrect. Expected (4, 3), but got {dA_prev.shape}"
    assert dW.shape == (2, 4), f"dW shape is incorrect. Expected (2, 4), but got {dW.shape}"
    assert db.shape == (2, 1), f"db shape is incorrect. Expected (2, 1), but got {db.shape}"

    # Manual calculation for comparison
    expected_dW = (1 / 3) * np.dot(dA, A_prev.T)
    expected_db = (1 / 3) * np.sum(dA, axis=1, keepdims=True)
    expected_dA_prev = np.dot(W.T, dA)

    assert np.allclose(dW, expected_dW), "dW calculation is incorrect"
    assert np.allclose(db, expected_db), "db calculation is incorrect"
    assert np.allclose(dA_prev, expected_dA_prev), "dA_prev calculation is incorrect"

    print("Test Case 2 passed!")

# Test Case 3: Invalid activation function
def test_case_3():
    np.random.seed(3)
    dA = np.random.randn(2, 2)
    cache = {"Z": np.random.randn(2, 2), "A_prev": np.random.randn(3, 2), "W": np.random.randn(2, 3)}

    try:
        backward_propagation(dA, cache, "invalid")
        assert False, "Expected ValueError was not raised"
    except ValueError as e:
        assert str(e) == "Hàm kích hoạt không được hỗ trợ", f"Unexpected error message: {str(e)}"

    print("Test Case 3 passed!")

# Test Case 4: ReLU activation with zero input
def test_case_4():
    dA = np.array([[1, 2], [3, 4]])
    A_prev = np.zeros((3, 2))
    W = np.array([[1, 2, 3], [4, 5, 6]])
    b = np.array([[0.1], [0.2]])
    Z = np.dot(W, A_prev) + b
    cache = {"Z": Z, "A_prev": A_prev, "W": W, "A": relu(Z)}

    dA_prev, dW, db = backward_propagation(dA, cache, "relu")

    assert np.allclose(dW, 0), "dW should be zero when A_prev is zero"
    assert np.allclose(db, np.array([[1.5], [3.5]])), "db calculation is incorrect for zero input"
    assert np.allclose(dA_prev, np.array([[13, 18], [17, 24], [21, 30]])), "dA_prev should be zero when all inputs are zero"

    print("Test Case 4 passed!")

# Test Case 5: Linear activation with large values
def test_case_5():
    np.random.seed(5)
    dA = np.random.randn(2, 3) * 1000
    A_prev = np.random.randn(4, 3) * 1000
    W = np.random.randn(2, 4)
    b = np.random.randn(2, 1)
    Z = np.dot(W, A_prev) + b
    cache = {"Z": Z, "A_prev": A_prev, "W": W, "A": Z}

    dA_prev, dW, db = backward_propagation(dA, cache, "linear")

    assert not np.any(np.isnan(dA_prev)), "dA_prev contains NaN values"
    assert not np.any(np.isnan(dW)), "dW contains NaN values"
    assert not np.any(np.isnan(db)), "db contains NaN values"

    print("Test Case 5 passed!")

# Test Case 6: ReLU activation with negative Z values
def test_case_6():
    dA = np.array([[1, 2], [3, 4]])
    A_prev = np.array([[-1, 2], [3, -4], [5, -6]])
    W = np.array([[1, 2, 3], [4, 5, 6]])
    b = np.array([[-0.1], [-0.2]])
    Z = np.dot(W, A_prev) + b
    cache = {"Z": Z, "A_prev": A_prev, "W": W, "A": relu(Z)}

    dA_prev, dW, db = backward_propagation(dA, cache, "relu")

    expected_dZ = dA * (Z > 0)
    assert np.allclose(np.dot(W.T, expected_dZ), dA_prev), "dA_prev calculation is incorrect for ReLU with negative Z values"

    print("Test Case 6 passed!")

# Test Case 7: Linear activation with small learning rate simulation
def test_case_7():
    np.random.seed(7)
    dA = np.random.randn(2, 3) * 0.01
    A_prev = np.random.randn(4, 3)
    W = np.random.randn(2, 4)
    b = np.random.randn(2, 1)
    Z = np.dot(W, A_prev) + b
    cache = {"Z": Z, "A_prev": A_prev, "W": W, "A": Z}

    dA_prev, dW, db = backward_propagation(dA, cache, "linear")

    assert np.all(np.abs(dW) < 0.1), "dW values are too large for small learning rate scenario"
    assert np.all(np.abs(db) < 0.1), "db values are too large for small learning rate scenario"

    print("Test Case 7 passed!")

# Test Case 8: ReLU activation with batch size 1
def test_case_8():
    dA = np.array([[1], [2]])
    A_prev = np.array([[3], [4], [5]])
    W = np.array([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]])
    b = np.array([[0.01], [0.02]])
    Z = np.dot(W, A_prev) + b
    cache = {"Z": Z, "A_prev": A_prev, "W": W, "A": relu(Z)}

    dA_prev, dW, db = backward_propagation(dA, cache, "relu")

    assert dA_prev.shape == (3, 1), f"dA_prev shape is incorrect. Expected (3, 1), but got {dA_prev.shape}"
    assert dW.shape == (2, 3), f"dW shape is incorrect. Expected (2, 3), but got {dW.shape}"
    assert db.shape == (2, 1), f"db shape is incorrect. Expected (2, 1), but got {db.shape}"

    print("Test Case 8 passed!")

test_case_2()
test_case_3()
test_case_4()
test_case_5()
test_case_6()
test_case_7()
test_case_8()

### END HIDDEN TESTS

Test Case 1 passed!
Test Case 2 passed!
Test Case 3 passed!
Test Case 4 passed!
Test Case 5 passed!
Test Case 6 passed!
Test Case 7 passed!
Test Case 8 passed!


# BÀI TẬP: Cập nhật tham số

## Gradient Descent

Gradient Descent là một phương pháp tối ưu hóa được sử dụng để giảm thiểu hàm mất mát của mô hình. Bằng cách cập nhật các tham số theo hướng của gradient âm của hàm mất mát, gradient descent giúp tìm kiếm các tham số tối ưu nhằm giảm thiểu giá trị của hàm mất mát.

### Các Bước Của Gradient Descent
1. **Khởi tạo tham số**: Bắt đầu với các giá trị ban đầu cho các tham số (thường là các giá trị ngẫu nhiên hoặc số không).
2. **Tính toán gradient**: Tính gradient của hàm mất mát đối với các tham số hiện tại.
3. **Cập nhật tham số**: Điều chỉnh các tham số bằng cách trừ đi một phần của gradient, tỷ lệ với tốc độ học (learning rate).
4. **Lặp lại**: Tiếp tục lặp lại các bước 2 và 3 cho đến khi đạt được hội tụ (giá trị hàm mất mát không thay đổi đáng kể) hoặc số lượng vòng lặp tối đa.

### Công Thức Gradient Descent
Giả sử hàm mất mát là $J(\theta)$, gradient descent sẽ cập nhật tham số \theta như sau:
\begin{equation*}
\theta := \theta - \alpha \frac{\partial J(\theta)}{\partial \theta}
\end{equation*}
trong đó:
- $\theta$ là vector tham số
- $\alpha$ là tốc độ học
- $\frac{\partial J(\theta)}{\partial \theta}$ là gradient của hàm mất mát đối với tham số $\theta$

In [11]:
def update_parameters(parameters, grads, learning_rate):
    """
    Cập nhật tham số sử dụng gradient descent với gradient clipping.

    Tham số:
    parameters -- dictionary chứa các tham số
    grads -- dictionary chứa các gradient
    learning_rate -- tốc độ học của quy tắc cập nhật gradient descent

    Trả về:
    parameters -- dictionary chứa các tham số đã được cập nhật
    """
    W = parameters["W"].astype(float)
    b = parameters["b"].astype(float)
    dW = grads["dW"]
    db = grads["db"]

    ### BEGIN SOLUTION
    W -= learning_rate * dW
    b -= learning_rate * db
    ### END SOLUTION

    parameters = {"W": W, "b": b}
    return parameters

In [12]:
# Tests 10 points.

# Test Case 1: Normal update without clipping
def test_case_1():
    np.random.seed(1)
    parameters = {
        "W": np.random.randn(3, 4),
        "b": np.random.randn(3, 1)
    }
    grads = {
        "dW": np.random.randn(3, 4) * 0.1,
        "db": np.random.randn(3, 1) * 0.1
    }
    learning_rate = 0.1

    original_W = parameters["W"].copy()
    original_b = parameters["b"].copy()

    updated_parameters = update_parameters(parameters, grads, learning_rate)

    assert np.allclose(updated_parameters["W"], original_W - learning_rate * grads["dW"]), "W update is incorrect"
    assert np.allclose(updated_parameters["b"], original_b - learning_rate * grads["db"]), "b update is incorrect"
    print("Test Case 1 passed!")

test_case_1()

### START HIDDEN TESTS

# Test Case 2: Update with zero gradients
def test_case_2():
    parameters = {
        "W": np.array([[1, 2], [3, 4]]),
        "b": np.array([[5], [6]])
    }
    grads = {
        "dW": np.zeros((2, 2)),
        "db": np.zeros((2, 1))
    }
    learning_rate = 0.1

    updated_parameters = update_parameters(parameters, grads, learning_rate)

    assert np.array_equal(updated_parameters["W"], parameters["W"]), "W should not change with zero gradients"
    assert np.array_equal(updated_parameters["b"], parameters["b"]), "b should not change with zero gradients"
    print("Test Case 2 passed!")

# Test Case 5: Update with very small learning rate
def test_case_3():
    np.random.seed(5)
    parameters = {
        "W": np.random.randn(3, 4),
        "b": np.random.randn(3, 1)
    }
    grads = {
        "dW": np.random.randn(3, 4),
        "db": np.random.randn(3, 1)
    }
    learning_rate = 1e-10

    original_W = parameters["W"].copy()
    original_b = parameters["b"].copy()

    updated_parameters = update_parameters(parameters, grads, learning_rate)

    assert np.allclose(updated_parameters["W"], original_W, atol=1e-8), "W update with small learning rate is incorrect"
    assert np.allclose(updated_parameters["b"], original_b, atol=1e-8), "b update with small learning rate is incorrect"
    print("Test Case 3 passed!")

# Run all test cases
test_case_2()
test_case_3()

### END HIDDEN TESTS

Test Case 1 passed!
Test Case 2 passed!
Test Case 3 passed!


# Chạy thử cập nhật tham số

Phần code dưới đây thực hiện một bước lan truyền xuôi và lan truyền ngược qua một lớp của mạng neural đơn giản, sau đó cập nhật các tham số trọng số và bias bằng cách sử dụng gradient descent. Mô hình này giả định đầu ra liên tục, với hàm mất mát là lỗi trung bình bình phương (MSE). Quá trình bao gồm các bước chính:

1. Khởi tạo dữ liệu ví dụ và tham số ban đầu.
2. Lan truyền xuôi để tính đầu ra của lớp và chi phí.
3. Lan truyền ngược để tính gradient của các tham số.
4. Cập nhật tham số bằng cách sử dụng gradient descent.

In [13]:
# Thiết lập dữ liệu ví dụ
np.random.seed(42)
X = np.random.randn(3, 5)  # 3 đặc trưng, 5 ví dụ
Y = np.random.randn(1, 5)  # Đầu ra liên tục cho MSE
W = np.random.randn(1, 3)  # 1 neuron, 3 đặc trưng đầu vào
b = np.zeros((1, 1))

# Lan truyền xuôi
Z = np.dot(W, X) + b
A = relu(Z)  # Giả sử hàm relu đã được định nghĩa

# Tính toán chi phí
cost = compute_cost(A, Y)
print("Chi phí ban đầu:", cost)

# Chuẩn bị cho lan truyền ngược
dA = A - Y  # Gradient của MSE đối với A

# Tạo cache
cache = {
    "Z": Z,
    "A_prev": X,
    "W": W,
    "A": A
}

# Thực hiện lan truyền ngược
dA_prev, dW, db = backward_propagation(dA, cache, "relu")

# Chuẩn bị cập nhật tham số
parameters = {"W": W, "b": b}
grads = {"dW": dW, "db": db}
learning_rate = 0.01

# Cập nhật tham số
updated_parameters = update_parameters(parameters, grads, learning_rate)

# In kết quả
print("\nGradient:")
print("dW:", grads["dW"])
print("db:", grads["db"])
print("\nTham số đã cập nhật:")
print("W:", updated_parameters["W"])
print("b:", updated_parameters["b"])

Chi phí ban đầu: 1.4685984771390461

Gradient:
dW: [[ 1.14172833 -0.2807227  -1.29120827]]
db: [[0.98140377]]

Tham số đã cập nhật:
W: [[ 1.45423149 -0.22296907  0.08044029]]
b: [[-0.00981404]]
